<a href="https://colab.research.google.com/github/vishwasjoshi2019/Stock_Market_Analysis/blob/main/RSI_based_SPX_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29089 sha256=0f1b32b1682958df1f0aed62a7533fdcc0a13f33a2be5ad56999f384c4ecfc1c
  Stored in directory: /root/.cache/pip/wheels/47/51/06/380dc516ea78621870b93ff65527c251afdfdc5fa9d7f4d248
Successfully built ta


In [40]:
import yfinance as yf
import pandas as pd
import ta

# Define the ticker symbol for S&P 500
ticker_symbol = "^GSPC"

# Download S&P 500 data from Yahoo Finance
snp_data = yf.download(ticker_symbol, period="10y")

# Function to calculate RSI using the ta library
def calculate_rsi(df, window=14):
    # Create an RSIIndicator instance
    rsi = ta.momentum.RSIIndicator(close=df['Close'], window=window)

    # Add RSI values to the DataFrame
    df['RSI'] = rsi.rsi()

    return df

# Calculate RSI
snp_data = calculate_rsi(snp_data)

# Define a function to check buy conditions
def check_condition(data):
    # Create an empty Series to store the buy signals
    signals = pd.Series(index=data.index, dtype=object)

    for i in range(len(data)):
        row = data.iloc[i]

        if (row['RSI'] > 60 and row['High'] >=data['High'].iloc[max(0, i - 150):i].max()):
          signals.iloc[i] = 'Buy'
        elif(row['RSI'] < 40 and row['Low'] <= data['Low'].iloc[max(0, i - 30):i].min()):
          signals.iloc[i] = 'Sell'
        else:
          signals.iloc[i] = 'No Signal'

    return signals

# Calculate buy signals for the entire DataFrame
snp_data['Signal'] = check_condition(snp_data)

# Filter rows with 'Buy' signals
signals_df = snp_data[(snp_data['Signal'] == 'Buy') | (snp_data['Signal'] == 'Sell')]


# Print the dates and corresponding signals
print("Buy Signals:")
print(signals_df[[ 'RSI', 'High', 'Low', 'Signal']])


[*********************100%%**********************]  1 of 1 completed
Buy Signals:
                  RSI         High          Low Signal
Date                                                  
2013-11-13  63.042459  1782.000000  1760.640015    Buy
2013-11-14  66.363129  1791.530029  1780.219971    Buy
2013-11-15  68.994400  1798.219971  1790.660034    Buy
2013-11-18  64.234467  1802.329956  1788.000000    Buy
2013-11-22  66.685095  1804.839966  1794.699951    Buy
...               ...          ...          ...    ...
2023-09-22  32.818414  4357.399902  4316.490234   Sell
2023-09-25  35.993463  4338.509766  4302.700195   Sell
2023-09-26  30.319021  4313.009766  4265.979980   Sell
2023-09-27  30.499954  4292.069824  4238.629883   Sell
2023-10-03  28.661478  4281.149902  4216.450195   Sell

[473 rows x 4 columns]


In [ ]:

# Create a DataFrame to store results
results = pd.DataFrame(columns=['Buy Date', 'Buy Close Price', 'Buy Date 90 Days Ahead', 'Close Price 90 Days Ahead', '3 Months Return'])

skip_rows = 90  # Define the number of rows to skip

i = 0
while i < len(snp_data):
    if snp_data['Signal'].iloc[i] == 'Buy':
        buy_date = snp_data.index[i]
        buy_close_price = snp_data['Close'].iloc[i]
        i += skip_rows  # Skip 90 rows
        if i >= len(snp_data):
            break

        # Calculate the 3 months ahead closing price and date
        three_months_later_date = snp_data.index[i]
        three_months_later_close_price = snp_data['Close'].iloc[i]
        three_months_return = three_months_later_close_price / buy_close_price - 1

        results = results.append({'Buy Date': buy_date, 'Buy Close Price': buy_close_price, 'Buy Date 90 Days Ahead': three_months_later_date, 'Close Price 90 Days Ahead': three_months_later_close_price, '3 Months Return': three_months_return}, ignore_index=True)
    else:
        i += 1

# Print the results
print("Returns for Buy Signals with Skipping in Between:")
print(results)

In [52]:
results

,Buy Date,Buy Close Price,Buy Date 90 Days Ahead,Close Price 90 Days Ahead,3 Months Return
0,2013-11-13,1782.000000,2014-03-26,1852.560059,0.039596
1,2014-04-01,1885.520020,2014-08-08,1931.589966,0.024434
2,2014-08-21,1992.369995,2014-12-30,2080.350098,0.044159
3,2015-02-13,2096.989990,2015-06-24,2108.580078,0.005527
4,2016-06-07,2112.129883,2016-10-13,2132.550049,0.009668
5,2016-11-21,2198.179932,2017-04-03,2358.840088,0.073088
6,2017-05-08,2399.379883,2017-09-14,2495.620117,0.040110
7,2017-09-14,2495.620117,2018-01-24,2837.540039,0.137008
8,2018-01-24,2837.540039,2018-06-04,2746.870117,-0.031954
9,2018-08-21,2862.959961,2018-12-31,2506.850098,-0.124385


In [54]:

# Create a DataFrame to store results for short conditions
short_results = pd.DataFrame(columns=['Short Date', 'Short Close Price', 'Cover Date 90 Days Ahead', 'Close Price 90 Days Ahead', '3 Months Return'])

skip_rows = 90  # Define the number of rows to skip

i = 0
while i < len(snp_data):
    if snp_data['Signal'].iloc[i] == 'Sell':
        short_date = snp_data.index[i]
        short_close_price = snp_data['Close'].iloc[i]
        i += skip_rows  # Skip 90 rows
        if i >= len(snp_data):
            break

        # Calculate the 3 months ahead closing price and date (Cover Date)
        cover_date = snp_data.index[i]
        cover_close_price = snp_data['Close'].iloc[i]
        three_months_return = short_close_price / cover_close_price - 1

        short_results = short_results.append({'Short Date': short_date, 'Short Close Price': short_close_price, 'Cover Date 90 Days Ahead': cover_date, 'Close Price 90 Days Ahead': cover_close_price, '3 Months Return': three_months_return}, ignore_index=True)
    else:
        i += 1

# Print the results for short conditions
print("Returns for Short Signals with Skipping in Between:")
print(short_results)

<ipython-input-54-7ce9e0e17ae3>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-54-7ce9e0e17ae3>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-54-7ce9e0e17ae3>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-54-7ce9e0e17ae3>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Returns for Short Signals with Skipping in Between:
   Short Date  Short Close Price Cover Date 90 Days Ahead  \
0  2014-02-03        1741.890015               2014-06-12   
1  2014-07-31        1930.670044               2014-12-08   
2  2014-12-15        1989.630005               2015-04-27   
3  2015-06-29        2057.639893               2015-11-04   
4  2015-12-11        2012.369995               2016-04-22   
5  2016-06-27        2000.540039               2016-11-02   
6  2016-11-02        2097.939941               2017-03-15   
7  2018-02-05        2648.939941               2018-06-14   
8  2018-10-10        2785.679932               2019-02-21   
9  2019-05-13        2811.870117               2019-09-19   
10 2020-02-24        3225.889893               2020-07-01   
11 2020-09-21        3281.060059               2021-01-29   
12 2021-09-20        4357.729980               2022-01-27   
13 2022-02-23        4225.500000               2022-07-05   
14 2022-09-01        3966.850098 

<ipython-input-54-7ce9e0e17ae3>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-54-7ce9e0e17ae3>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-54-7ce9e0e17ae3>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-54-7ce9e0e17ae3>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-54-7ce9e0e17ae3>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-54-7ce9e0e17ae3>:20: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Us

In [55]:
short_results

,Short Date,Short Close Price,Cover Date 90 Days Ahead,Close Price 90 Days Ahead,3 Months Return
0,2014-02-03,1741.890015,2014-06-12,1930.109985,-0.097518
1,2014-07-31,1930.670044,2014-12-08,2060.310059,-0.062923
2,2014-12-15,1989.630005,2015-04-27,2108.919922,-0.056564
3,2015-06-29,2057.639893,2015-11-04,2102.310059,-0.021248
4,2015-12-11,2012.369995,2016-04-22,2091.580078,-0.037871
5,2016-06-27,2000.540039,2016-11-02,2097.939941,-0.046426
6,2016-11-02,2097.939941,2017-03-15,2385.260010,-0.120456
7,2018-02-05,2648.939941,2018-06-14,2782.489990,-0.047997
8,2018-10-10,2785.679932,2019-02-21,2774.879883,0.003892
9,2019-05-13,2811.870117,2019-09-19,3006.790039,-0.064827


In [39]:

# Create a candlestick chart
fig = go.Figure(data=[go.Candlestick(
    x=snp_data.index,
    open=snp_data['Open'],
    high=snp_data['High'],
    low=snp_data['Low'],
    close=snp_data['Close'],
    name='Candlesticks'
)])

# Create a subplot for the closing price
fig.add_trace(go.Scatter(x=snp_data.index, y=snp_data['Close'], mode='lines', name='Closing Price', yaxis='y2'))

# Create a subplot for RSI
fig.add_trace(go.Scatter(x=snp_data.index, y=snp_data['RSI'], mode='lines', name='RSI', yaxis='y3'))

# Update the layout to include multiple y-axes and arrange subplots
fig.update_layout(
    xaxis_rangeslider_visible=False,
    yaxis=dict(title='Candlestick Chart', domain=[0.7, 1]),
    yaxis2=dict(title='Closing Price', domain=[0.45, 0.65]),
    yaxis3=dict(title='RSI', domain=[0.2, 0.4]),
    title='S&P 500 Candlestick Chart, Closing Price, and RSI'
)

# Show the interactive plot
fig.show()